# Part 2: Feature Engineering

Author: Julianne <br>
Created: 1/28/2018<br>
Updated: 2/26/2018<br>

Step 1: Load data. <br>

Step 2: From video duration.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) hours, mins, secs, total_min<br>

Step 2: From time at which video was published.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) day, week, month, year<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) season <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) weekday/weekend <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5) morning/afternoon <br>

Step 3: From video title.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) length of title <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) vlog <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) vlogmas <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) series <br>

Step 4: From video statistics.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) likes/viewCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) dislikes/viewCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) comments/viewCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) (likes+dislikes)/viewCount <br>

Step 4: Create dataframe info for unique channelIds.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1) yearBegan <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2) yearCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3) vidCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4) viewMax <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;5) viewAvg <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;6) viewChange <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;7) viewLabel <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;8) stats_subscriberCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;9) posInteract <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;10) negInteract <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;11) comInteract <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;12) minsTotal <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;13) channelSize <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;14) plistCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;15) vlogCount <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;16) dayofweek <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;17) timeofday <br>

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Software versions
Python 3.6.3 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 5.1.0
OS Darwin 16.1.0 x86_64 i386 64bit
numpy 1.13.3
scipy 0.18.1
matplotlib 2.0.0
pandas 0.19.2
Tue Feb 27 11:48:13 2018 EST

In [4]:
from __future__ import division
from datetime import datetime 
import requests
from lxml import html, etree
import json
from textblob import TextBlob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
import timeit
from threading import Thread
import queue

#never print matching warnings
warnings.filterwarnings('ignore') 

#set how pandas displays data
pd.options.display.max_columns = 100
pd.options.display.max_rows = 35
pd.options.display.width = 120

#change the max # of rows pandas will display
pd.options.display.max_rows = 9000

#set to display all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#import seaborn to draw pretty graphs
import seaborn as sns
sns.set_style('whitegrid')

#name of files
name='Youtube_cat26_byview_2017_'

# Step 1 : Load data

In [15]:
#save as csv
filename='Youtube_Part1_cat26_byview_2017_RawData_Final.csv'
data=pd.read_csv(filename)
del data['Unnamed: 0']
data.head(1)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,posInteract,negInteract,anyInteract,comInteract,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,vlog,vlogmas,title_length,description_length,title_channel,month_name,season,dayofweek,dayofweek2,type_of_day,hour,timeofday,series,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,2.098313,0.093789,2.192102,0.230358,0,5,47,5,347,2017,6,26,29,180,no,no,45,199,excludes_CT,Jun,summer,3,Thurs,weekday,17,afternoon,no,0,False,7648588,873,1480889507,17,1.0


# Step 2: From 'videoDuration'

## 1) Extract hours, mins, secs, total_min

In [22]:
def convert_time(df):
    
    df['hours']=0
    df['minutes']=0
    df['seconds']=0
    df['total_minutes']=0
    df['total_seconds']=0

    for i in range(len(df)): # using 0 in range(0,len(df)) is not necessary
        
        #print(i)
        
        hours=0
        minutes=0
        seconds=0
        
        curr_str=df['videoDuration'][i]
        
        split_str1=curr_str.split('T')
        curr_str=split_str1[1]

        if 'H' in curr_str:
            split_str =curr_str.split('H')
            hours     =int(split_str[0])
            curr_str  =split_str[1]
        
        if 'M' in curr_str:
            split_str =curr_str.split('M')
            minutes   =int(split_str[0])
            curr_str  =split_str[1]

        if 'S' in curr_str:
            split_str = curr_str.split('S')
            seconds   = int(split_str[0])
            
        df['hours'][i]=hours
        df['minutes'][i]=minutes
        df['seconds'][i]=seconds
        df['total_minutes'][i]=hours*60+minutes
        df['total_seconds'][i]=hours*60*60+minutes*60+seconds
    
    return df

In [23]:
def convert_hours(duration):
    hours=0
    minutes=0
    seconds=0
    if 'PT' in duration:
        duration=duration.split('PT')[1]
    if 'H' in duration:
        hours = duration.split('H')[0]
        hours = int(hours)
    return hours

def convert_minutes(duration):
    hours=0
    minutes=0
    seconds=0
    if 'PT' in duration:
        duration=duration.split('PT')[1]
    if 'H' in duration:
        duration = duration.split('H')[1]
    if 'M' in duration:
        minutes = duration.split('M')[0]
        minutes = int(minutes)
    return minutes

def convert_seconds(duration):  
    hours=0
    minutes=0
    seconds=0
    if 'PT' in duration:
        duration=duration.split('PT')[1]
    if 'H' in duration:
        duration = duration.split('H')[1]
    if 'M' in duration:
        duration = duration.split('M')[1]  
    if 'S' in duration:
        seconds = duration.split('S')[0]
        seconds = int(seconds)
    return seconds

def convert_totalmins(df):
    hours=df[0]
    minutes=df[1]
    total_mins=hours*60+minutes
    return total_mins

def convert_totalsecs(df):
    hours=df[0]
    minutes=df[1]
    seconds=df[2]
    total_mins=hours*60*60+minutes*60+seconds
    return total_mins

In [24]:
data['hours'] = data['videoDuration'].apply(func=convert_hours)
data['minutes'] = data['videoDuration'].apply(func=convert_minutes)
data['seconds'] = data['videoDuration'].apply(func=convert_seconds)
data['total_mins'] = data[['hours','minutes']].apply(func=convert_totalmins, axis=1)
data['total_secs'] = data[['hours','minutes','seconds']].apply(func=convert_totalsecs, axis=1)

data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457


# Step 2: From 'publishedAt'

## 1) Determine day, week, month, year

In [25]:
from datetime import datetime

def determine_year(data):
    return pd.to_datetime(data).year

def determine_month(data):
    return pd.to_datetime(data).month

def determine_week(data):
    return pd.to_datetime(data).week

def determine_day(data):
    return pd.to_datetime(data).day

def determine_dayofyear(data):
    return pd.to_datetime(data).dayofyear

data['year'] = data['publishedAt'].apply(func=determine_year)
data['month'] = data['publishedAt'].apply(func=determine_month)
data['week'] = data['publishedAt'].apply(func=determine_week)
data['day'] = data['publishedAt'].apply(func=determine_day)
data['dayofyear'] = data['publishedAt'].apply(func=determine_dayofyear)

data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180


In [26]:
def determine_nameofmonth(month):
    
    if month == 1:
        curr_month='Jan'
    if month == 2:
        curr_month='Feb'
    if month == 3:
        curr_month='Mar'
    if month == 4:
        curr_month='Apr'
    if month == 5:
        curr_month='May'
    if month == 6:
        curr_month='Jun'
    if month == 7:
        curr_month='Jul'
    if month == 8:
        curr_month='Aug'
    if month == 9:
        curr_month='Sep'
    if month == 10:
        curr_month='Oct'
    if month == 11:
        curr_month='Nov'
    if month == 12:
        curr_month='Dec'

    return curr_month

data['month_name'] = data['month'].apply(func=determine_nameofmonth)

In [27]:
def determine_codeofday(data):
    return pd.to_datetime(data).dayofweek

data['dayofweek'] = data['publishedAt'].apply(func=determine_codeofday)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3


In [28]:
def determine_nameofday(output):
    if output==0:
        return 'Mon'
    if output==1:
        return 'Tues'
    if output==2:
        return 'Wed'
    if output==3:
        return 'Thurs'
    if output==4:
        return 'Fri'
    if output==5:
        return 'Sat'
    if output==6:
        return 'Sun'

data['dayofweek2'] = data['dayofweek'].apply(func=determine_nameofday)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs


## 2) Determine season

In [29]:
def determine_season(month):
    if month == 'Dec' or month == 'Jan' or month == 'Feb':
        season='winter'
    if month == 'Mar' or month == 'Apr' or month == 'May':
        season='spring'
    if month == 'Jun' or month == 'Jul' or month == 'Aug':
        season='summer'
    if month == 'Sep' or month == 'Oct' or month == 'Nov':
        season='Fall'
    return season

data['season'] = data['month_name'].apply(func=determine_season)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer


## 3) Determine if weekday or not

In [30]:
def determine_weekday(day_of_week):
    if day_of_week == 'Mon' or day_of_week == 'Tues' or day_of_week == 'Wed' or day_of_week == 'Thurs' or day_of_week == 'Fri':
        return 'weekday'
    if day_of_week == 'Sat' or day_of_week == 'Sun':
        return 'weekend'  
    
data['type_of_day']=data['dayofweek2'].apply(func=determine_weekday)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday


## 4) Determine if published in morning or afternoon

In [31]:
def determine_hour(data):
    return pd.to_datetime(data).hour

data['hour'] = data['publishedAt'].apply(func=determine_hour)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11


In [32]:
def determine_timeofday(hour):
    if hour<12:
        return 'morning'
    else:
        return 'afternoon'
    
data['timeofday'] = data['hour'].apply(func=determine_timeofday)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning


# Step 4: From 'title'

## 1) Characterize if video is vlog or not

In [33]:
def determine_vlog(title):
    if 'vlog' in title.lower():
        return 'yes'
    else:
        return 'no'
    
data['vlog'] = data['title'].apply(func=determine_vlog)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no


## 2) Characterize if video is part of vlogmas

In [34]:
def vlogmas(title):
    if 'vlogmas' in title.lower():
        return 'yes'
    else:
        return 'no'
    
data['vlogmas'] = data['title'].apply(func=vlogmas)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no,no


## 3) Determine length of video title

In [35]:
def determine_titlelength(title):
    return len(title)

data['title_length'] = data['title'].apply(func=determine_titlelength)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no,no,37


## 4) Determine if video is part of a series

In [36]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

def determine_series(title):
    if ' part ' in title.lower() and hasNumbers(title):
        return 'yes'
    else:
        return 'no' 
    
data['series']=data['title'].apply(func=determine_series)
data.head(2)

#note not 100% accurate but will provide good estimate on if this variable is impactful or not

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length,series
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45,no
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no,no,37,no


# Step 5: From 'description'

### 1) Determine length of description

In [37]:
def determine_desclength(description):
    if type(description)==str:
        return len(description)
    else:
        return 0
    
data['description_length'] = data['description'].apply(func=determine_desclength)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length,series,description_length
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45,no,199
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no,no,37,no,199


# Step 6: From video statistics ('likeCount','dislikeCount', 'commentCount')

### 1) Determine amount of viewere interaction

In [38]:
def determine_posInteraction(df):
    viewCount=df[0]
    likeCount=df[1]
    interaction=(likeCount/viewCount)*100
    return interaction

def determine_negInteraction(df):
    viewCount=df[0]
    dislikeCount=df[1]
    interaction=(dislikeCount/viewCount)*100
    return interaction

def determine_anyInteraction(df):
    viewCount=df[0]
    dislikeCount=df[1]
    likeCount=df[2]
    interaction=((dislikeCount+likeCount)/viewCount)*100
    return interaction

def determine_comments(df):
    viewCount=df[0]
    commentCount=df[1]
    interaction=(commentCount/viewCount)*100
    return interaction

data['posInteract'] = data[['viewCount','likeCount']].apply(func=determine_posInteraction,axis=1)
data['negInteract'] = data[['viewCount','dislikeCount']].apply(func=determine_negInteraction,axis=1)
data['anyInteract'] = data[['viewCount','dislikeCount','likeCount']].apply(func=determine_anyInteraction,axis=1)
data['comInteract'] = data[['viewCount','commentCount']].apply(func=determine_comments,axis=1)
data.head(2)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length,series,description_length,posInteract,negInteract,anyInteract,comInteract
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7648588,873,1480889507,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45,no,199,2.098313,0.093789,2.192102,0.230358
1,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,v2g9xuJynVY,8 Tricks Bosses Use to Steal Your Pay,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T11:00:05.000Z,110,26,26,PT7M37S,77.0,67.0,1007.0,38640.0,0,False,7648588,873,1480889507,17,NaN,0,7,37,7,457,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,11,morning,no,no,37,no,199,2.606108,0.173395,2.779503,0.199275


# Step 5: Final dataframe with channel information

Remove channels with no subscribers

In [39]:
#channels with no subscribers
data[data['stats_subscriberCount']==0].shape[0]
#remove channels without subscribers
data=data[data['stats_subscriberCount']>0]
#channels with no subscribers
data[data['stats_subscriberCount']==0].shape[0]

0

0

Remove videos without views. These videos are most likely unlisted and would not contribute toward channel growth.

In [41]:
#remove videos with no views
data[data['viewCount']==0].shape[0]
#remove channels without subscribers
data=data[data['viewCount']>0]
#channels with no subscribers
data[data['viewCount']==0].shape[0]

0

0

Create new dataframe for each unique channel.<br><br>
Add following variables:<br>
1) 'yearBegan' - define year at which channel was started<br>
2) 'yearCount' - calculate number of years during which channel has been active (posted videos)

In [42]:
col=['channelTitle','yearBegan','yearCount']
channels=data.channelTitle.unique()
channelData=pd.DataFrame(columns=col, index=range(len(channels)))
channels=data.channelTitle.unique()
row=-1
for channel in channels:
    row=row+1
    df_begin =data[data['channelTitle']==channel].reset_index()
    channelData['channelTitle'][row]=channel
    channelData['yearBegan'][row]=df_begin['year'].min()
    channelData['yearCount'][row]=df_begin['year'].max()-df_begin['year'].min()
channelData.head(2)

,channelTitle,yearBegan,yearCount
0,BRIGHT SIDE,2017,0
1,ATHLEAN-X™,2010,7


### Determine number of videos posted for each channel

Add following variables:<br>
1) 'vidCountAll' - calculate number of videos posted in lifetime of channel<br>
2) 'vidCountBegin' - calculate number of videos posted in first year of channel<br>
3) 'vidCount2015' - calculate number of videos posted during 2015<br>
4) 'vidCount2016' - calculate number of videos posted during 2016<br>
5) 'vidCount2017' - calculate number of videos posted during 2017<br>

In [25]:
channelData['vidCountAll']=0
channelData['vidCountBegin']=0
channelData['vidCount2015']=0
channelData['vidCount2016']=0
channelData['vidCount2017']=0

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()

    channelData['vidCountAll'][row]=df_all.shape[0]
    channelData['vidCountBegin'][row]=df_begin.shape[0]
    
    if channelData.yearBegan[i]<=2015:
        channelData['vidCount2015'][row]=df_2015.shape[0]
    if channelData.yearBegan[i]<=2016:
        channelData['vidCount2016'][row]=df_2016.shape[0]
    if channelData.yearBegan[i]<=2017:
        channelData['vidCount2017'][row]=df_2017.shape[0]
    
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100
1,ATHLEAN-X™,2010,7,628,53,100,100,100


### Determine number of views for videos posted by each channel

Add following variables:<br>
1) 'viewMaxAll' - maximum viewCount from a video posted anytime in lifetime of channel<br>
2) 'viewMaxBegin' - maximum viewCount from a video posted  in first year<br>
3) 'viewMax2015' - maximum viewCount from a video posted  during 2015<br>
4) 'viewMax2016' - maximum viewCount from a video posted  during 2016<br>
5) 'viewMax2017' - maximum viewCount from a video posted  during 2017<br>

6) 'viewAvgAll' - average viewCount of videos posted anytime in lifetime of channel<br>
7) 'viewAvgBegin' - average viewCount of videos posted during first year<br>
8) 'viewAvg2015' - average viewCount of videos posted during 2015<br>
9) 'viewAvg2016' - average viewCount of videos posted during 2016<br>
10) 'viewAvg2017' - average viewCount of videos posted during 2017<br>

In [26]:
channelData['viewMaxAll']=0
channelData['viewMaxBegin']=0
channelData['viewMax2015']=0
channelData['viewMax2016']=0
channelData['viewMax2017']=0

channelData['viewAvgAll']=0
channelData['viewAvgBegin']=0
channelData['viewAvg2015']=0
channelData['viewAvg2016']=0
channelData['viewAvg2017']=0

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()

    channelData['viewMaxAll'][row]=df_all['viewCount'].max()
    channelData['viewAvgAll'][row]=df_all['viewCount'].mean()
    
    channelData['viewMaxBegin'][row]=df_all['viewCount'].max()
    channelData['viewAvgBegin'][row]=df_all['viewCount'].mean()
    
    if channelData.yearBegan[i]<=2015:
        channelData['viewMax2015'][row]=df_2015['viewCount'].max()
        channelData['viewAvg2015'][row]=df_2015['viewCount'].mean()
        
    if channelData.yearBegan[i]<=2016:  
        channelData['viewMax2016'][row]=df_2016['viewCount'].max()
        channelData['viewAvg2016'][row]=df_2016['viewCount'].mean()
        
    if channelData.yearBegan[i]<=2017:
        channelData['viewMax2017'][row]=df_2017['viewCount'].max()
        channelData['viewAvg2017'][row]=df_2017['viewCount'].mean()
        
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0


### Determine if the views for each channel increased or decreased overtime

In a new dataframe, determine the following:<br>
1) 'channelTitle' - all unique channels <br>
2) 'year' - years in which channels posted at least one video <br>
3) 'viewCount' - average viewCount for the videos posted in each year<br>
4) 'change' - change in average viewCount from the previous year<br>

In [27]:
col=['channelTitle','year','viewCount','change']
channels=data.channelTitle.unique()
years=data.year.unique()
curr_range=len(channels)*len(years)
df2=pd.DataFrame(columns=col, index=range(curr_range))

row=-1
for channel in channels:
    subdata=data[data['channelTitle']==channel].sort(['year'])
    years=subdata.year.unique()
    for i in range(len(years)):
        viewCount=subdata[subdata['year']==years[i]]['viewCount'].mean()
        row=row+1
        df2['channelTitle'][row]=channel
        df2['year'][row]=years[i]
        df2['viewCount'][row]=viewCount
        if i==0:
            df2['change'][row]=0
        else:
            df2['change'][row]=viewCount-df2['viewCount'][row-1]

df2=df2[df2['channelTitle'].notnull()]
df2.head(10)

,channelTitle,year,viewCount,change
0,BRIGHT SIDE,2017,1.61184e+06,0
1,ATHLEAN-X™,2010,363391,0
2,ATHLEAN-X™,2011,277832,-85558.4
3,ATHLEAN-X™,2012,388071,110239
4,ATHLEAN-X™,2013,1.01673e+06,628660
5,ATHLEAN-X™,2014,954611,-62119.9
6,ATHLEAN-X™,2015,836367,-118244
7,ATHLEAN-X™,2016,849245,12877.7
8,ATHLEAN-X™,2017,842108,-7136.58
9,5-Minute Crafts,2016,59643,0


Add following variable:<br>
1) 'viewChangeAll' - change in average viewCount from first to last year of channel <br>
2) 'viewChange2015' - change in average viewCount from 2014 to 2015 <br>
3) 'viewChange2016' - change in average viewCount from 2015 to 2016<br>
4) 'viewChange2017' - change in average viewCount from 2016 to 2017<br>

In [28]:
channelData['viewChangeAll']=0
channelData['viewChange2015']=0
channelData['viewChange2016']=0
channelData['viewChange2017']=0

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =df2[df2['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()
    
    channelData['viewChangeAll'][row]=df_all['change'].sum()
    if channelData.yearBegan[i]<=2015:
        channelData['viewChange2015'][row]=df_2015['change'].sum()
    if channelData.yearBegan[i]<=2016:
        channelData['viewChange2016'][row]=df_2016['change'].sum()
    if channelData.yearBegan[i]<=2017:
        channelData['viewChange2017'][row]=df_2017['change'].sum()
    
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136


Add following catagorical variable:<br>
1) 'viewLabelAll' - did change in viewCount from first to last year of channel increase or decrease?<br>
2) 'viewLabel2015' - did change in viewCount from 2014 to 2015 increase or decrease? <br>
3) 'viewLabel2016' - did change in viewCount from 2015 to 2016 increase or decrease?<br>
4) 'viewLabel2017' - did change in viewCount from 2016 to 2017 increase or decrease?<br>

If viewCount increased > 100, channel is labled as 'growing'<br>
If viewCount decreased < -100, channel is labled as 'growing'<br>
If viewCount did not increase or decrease by specified threshold, channel is labeled as 'same'<br>

In [29]:
def class_grow(data):
    output='same'
    if data<-100:
        output='shrinking'
    if data>100:
        output='growing'
    return output

channelData['viewLabelAll']=channelData['viewChangeAll'].apply(func=class_grow)
channelData['viewLabel2015']=channelData['viewChange2015'].apply(func=class_grow)
channelData['viewLabel2016']=channelData['viewChange2016'].apply(func=class_grow)
channelData['viewLabel2017']=channelData['viewChange2017'].apply(func=class_grow)
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking


Add following numerical version of the previous catagorical variable:<br>
1) 'viewLabelAll_2' - did change in viewCount from first to last year of channel increase or decrease?<br>
2) 'viewLabel2015_2' - did change in viewCount from 2014 to 2015 increase or decrease? <br>
3) 'viewLabel2016_2' - did change in viewCount from 2015 to 2016 increase or decrease?<br>
4) 'viewLabel2017_2' - did change in viewCount from 2016 to 2017 increase or decrease?<br>

If viewCount increased > 100, channel is assigned as 1<br>
If viewCount decreased < -100, channel is assigned as 0<br>
If viewCount did not increase or decrease by specified threshold, channel is assigned as 2<br>

In [30]:
def binary_func(data):
    output=-1
    if data=='shrinking':
        output = 0
    if data=='growing':
        output = 1
    if data=='same':
        output = 2
    return output
    
channelData['viewLabelAll_2']=channelData['viewLabelAll'].apply(func=binary_func)
channelData['viewLabel2015_2']=channelData['viewLabel2015'].apply(func=binary_func)
channelData['viewLabel2016_2']=channelData['viewLabel2016'].apply(func=binary_func)
channelData['viewLabel2017_2']=channelData['viewLabel2017'].apply(func=binary_func)

channelData.head(1)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2


### Add subscription info

Add following variable:<br>
1) 'stats_subscriberCount' - number of subscribers at time data was taken from YouTube API<br>

In [31]:
channels=data.drop_duplicates(subset='channelTitle')
col=['channelTitle','stats_subscriberCount']
channelData=channelData.merge(channels[col], on='channelTitle',how='left')
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450


## Add information on viewer interaction

Add following variables:<br>
1) 'posInteractAll' - average number of likes per view for videos posted in lifetime of channel<br>
2) 'posInteractBegin' - average number of likes per view for videos posted in first year of channel<br>
3) 'posInteract2015' - average number of likes per view for videos posted in 2015<br>
4) 'posInteract2016' - average number of likes per view for videos posted in 2016<br>
5) 'posInteract2017' - average number of likes per view for videos posted in 2017<br>

1) 'negInteractAll' - average number of dislikes per view for videos posted in lifetime of channel<br>
2) 'negInteractBegin' - average number of dislikes per view for videos posted in first year of channel<br>
3) 'negInteract2015' - average number of dislikes per view for videos posted in 2015<br>
4) 'negInteract2016' - average number of dislikes per view for videos posted in 2016<br>
5) 'negInteract2017' - average number of dislikes per view for videos posted in 2017<br>

1) 'comInteractAll' - average number of comments per view for videos posted in lifetime of channel<br>
2) 'comInteractBegin' - average number of comments per view for videos posted in first year of channel<br>
3) 'comInteract2015' - average number of comments per view for videos posted in 2015<br>
4) 'comInteract2016' - average number of comments per view for videos posted in 2016<br>
5) 'comInteract2017' - average number of comments per view for videos posted in 2017<br>

In [32]:
channels=list(channelData['channelTitle'].unique())

channelData['posInteractAll']=0
channelData['negInteractAll']=0
channelData['comInteractAll']=0

channelData['posInteractBegin']=0
channelData['negInteractBegin']=0
channelData['comInteractBegin']=0

channelData['posInteract2015']=0
channelData['negInteract2015']=0
channelData['comInteract2015']=0

channelData['posInteract2016']=0
channelData['negInteract2016']=0
channelData['comInteract2016']=0

channelData['posInteract2017']=0
channelData['negInteract2017']=0
channelData['comInteract2017']=0

#ensure that columns contain floats
channelData['posInteractAll']=channelData['posInteractAll'].astype(float)
channelData['negInteractAll']=channelData['negInteractAll'].astype(float)
channelData['comInteractAll']=channelData['comInteractAll'].astype(float)
channelData['posInteractBegin']=channelData['posInteractBegin'].astype(float)
channelData['negInteractBegin']=channelData['negInteractBegin'].astype(float)
channelData['comInteractBegin']=channelData['comInteractBegin'].astype(float)
channelData['posInteract2015']=channelData['posInteract2015'].astype(float)
channelData['negInteract2015']=channelData['negInteract2015'].astype(float)
channelData['comInteract2015']=channelData['comInteract2015'].astype(float)
channelData['posInteract2016']=channelData['posInteract2016'].astype(float)
channelData['negInteract2016']=channelData['negInteract2016'].astype(float)
channelData['comInteract2016']=channelData['comInteract2016'].astype(float)
channelData['posInteract2017']=channelData['posInteract2017'].astype(float)
channelData['negInteract2017']=channelData['negInteract2017'].astype(float)
channelData['comInteract2017']=channelData['comInteract2017'].astype(float)

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]    
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()
    
    channelData['posInteractAll'][row]=df_all['posInteract'].mean()
    channelData['negInteractAll'][row]=df_all['negInteract'].mean()
    channelData['comInteractAll'][row]=df_all['comInteract'].mean()
    
    channelData['posInteractBegin'][row]=df_begin['posInteract'].mean()
    channelData['negInteractBegin'][row]=df_begin['negInteract'].mean()
    channelData['comInteractBegin'][row]=df_begin['comInteract'].mean()
    
    if channelData.yearBegan[i]<=2015:
        channelData['posInteract2015'][row]=df_2015['posInteract'].mean()
        channelData['negInteract2015'][row]=df_2015['negInteract'].mean()
        channelData['comInteract2015'][row]=df_2015['comInteract'].mean()
    
    if channelData.yearBegan[i]<=2016:
        channelData['posInteract2016'][row]=df_2016['posInteract'].mean()
        channelData['negInteract2016'][row]=df_2016['negInteract'].mean()
        channelData['comInteract2016'][row]=df_2016['comInteract'].mean()
        
    if channelData.yearBegan[i]<=2017:
        channelData['posInteract2017'][row]=df_2017['posInteract'].mean()
        channelData['negInteract2017'][row]=df_2017['negInteract'].mean()
        channelData['comInteract2017'][row]=df_2017['comInteract'].mean()
    
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304


## Add information on length of videos

Add following variables:<br>
1) 'minsTotalAll' - average number of likes per view for videos posted in lifetime of channel<br>
2) 'minsTotalBegin' - average number of likes per view for videos posted in first year of channel<br>
3) 'minsTotal2015' - average number of likes per view for videos posted in 2015<br>
4) 'minsTotal2016' - average number of likes per view for videos posted in 2016<br>
5) 'minsTotal2017' - average number of likes per view for videos posted in 2017<br>

In [33]:
channels=list(channelData['channelTitle'].unique())

channelData['minsTotalAll']=0
channelData['minsTotalBegin']=0
channelData['minsTotal2015']=0
channelData['minsTotal2016']=0
channelData['minsTotal2017']=0

channelData['minsTotalAll']=channelData['minsTotalAll'].astype(float)
channelData['minsTotalBegin']=channelData['minsTotalBegin'].astype(float)
channelData['minsTotal2015']=channelData['minsTotal2015'].astype(float)
channelData['minsTotal2016']=channelData['minsTotal2016'].astype(float)
channelData['minsTotal2017']=channelData['minsTotal2017'].astype(float)

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]    
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()
    
    channelData['minsTotalAll'][row]=df_all['total_mins'].mean()
    channelData['minsTotalBegin'][row]=df_begin['total_mins'].mean()
    
    if channelData.yearBegan[i]<=2015:
        channelData['minsTotal2015'][row]=df_2015['total_mins'].mean()
    
    if channelData.yearBegan[i]<=2016:
        channelData['minsTotal2016'][row]=df_2016['total_mins'].mean()
        
    if channelData.yearBegan[i]<=2017:
        channelData['minsTotal2017'][row]=df_2017['total_mins'].mean()
    
channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285,8.380000,8.380000,0.00,0.00,8.38
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304,6.308917,5.811321,5.18,5.75,6.67


## Classify size of the channel

Add following variable:<br>
1) 'channelSize2' - is the channel large or small?<br>

In [34]:
def channelSize2(data):
    if data>=100000:
        return 'large'
    if data< 100000:
        return 'small'
    
channelData['channelSize2']=channelData['stats_subscriberCount'].apply(func=channelSize2)
channelData.head(1)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017,channelSize2
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.0,0.0,0.0,0.0,0.0,0.0,1.881192,0.138871,0.227285,8.38,8.38,0.0,0.0,8.38,large


## Add playlist information

Add following variables:<br>
1) 'minsTotalAll' - number of playlists posted in lifetime of channel<br>
2) 'plistCountBegin' - number of playlists posted in first year of channel<br>
3) 'plistCount2015' - number of playlists posted in 2015<br>
4) 'plistCount2016' - number of playlists posted in 2016<br>
5) 'plistCount2017' - number of playlists posted in 2017<br>

In [35]:
channels=list(channelData['channelTitle'].unique())

channelData['plistCountAll']=0
channelData['plistCountBegin']=0
channelData['plistCount2015']=0
channelData['plistCount2016']=0
channelData['plistCount2017']=0

channelData['plistCountAll']=channelData['plistCountAll'].astype(float)
channelData['plistCountBegin']=channelData['plistCountBegin'].astype(float)
channelData['plistCount2015']=channelData['plistCount2015'].astype(float)
channelData['plistCount2016']=channelData['plistCount2016'].astype(float)
channelData['plistCount2017']=channelData['plistCount2017'].astype(float)

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()
    
    if df_all.shape[0]>0:
        channelData['plistCountAll'][row]=df_all['num_of_playlists'][0]
        channelData['plistCountBegin'][row]=df_begin['num_of_playlists'][0]
    
    if df_2015.shape[0]>0:
        channelData['plistCount2015'][row]=df_2015['num_of_playlists'][0]
    
    if df_2016.shape[0]>0:
        channelData['plistCount2016'][row]=df_2016['num_of_playlists'][0]
        
    if df_2017.shape[0]>0:
        channelData['plistCount2017'][row]=df_2017['num_of_playlists'][0]
    
channelData.head(5)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017,channelSize2,plistCountAll,plistCountBegin,plistCount2015,plistCount2016,plistCount2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285,8.380000,8.380000,0.000000,0.000000,8.38000,large,17.0,17.0,0.0,0.0,17.0
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304,6.308917,5.811321,5.180000,5.750000,6.67000,large,21.0,21.0,21.0,21.0,21.0
2,5-Minute Crafts,2016,1,150,50,0,50,100,26082766,26082766,0.0,313697.0,26082766.0,2409232,2409232,0.0,59643.0,3584027.0,3524384,0,0,3524384,growing,same,same,growing,1,2,2,1,14109560,0.907235,0.066111,0.052616,0.807436,0.040443,0.031656,0.000000,0.000000,0.000000,0.807436,0.040443,0.031656,0.957135,0.078944,0.063096,11.566667,0.500000,0.000000,0.500000,17.10000,large,7.0,7.0,0.0,7.0,7.0
3,Jaclyn Hill,2011,6,301,4,60,48,32,15429406,15429406,6042557.0,6277132.0,9018504.0,1312493,1312493,1532521.0,1624083.0,2155248.0,1803220,523383,91561,531164,growing,growing,growing,growing,1,1,1,1,4919905,3.343569,0.068461,0.412224,1.691926,0.028710,0.150864,3.920757,0.058560,0.530401,4.372235,0.092529,0.606775,4.611552,0.172237,0.566143,14.631229,6.000000,16.016667,17.458333,20.65625,large,14.0,14.0,14.0,14.0,14.0
4,Rosanna Pansino,2011,6,523,13,91,96,100,176018933,176018933,51699304.0,20856918.0,13306227.0,4297539,4297539,6056932.0,3253271.0,1720934.0,274088,-1394894,-2803660,-1532337,growing,shrinking,shrinking,shrinking,1,0,0,0,9643380,1.736919,0.031885,0.399245,0.825019,0.034042,0.182375,1.611941,0.029157,0.411249,2.291382,0.029794,0.471068,3.108589,0.040637,0.572232,8.005736,4.923077,8.164835,8.354167,7.77000,large,5.0,5.0,5.0,5.0,5.0


## Add vlog information

Add following variables:<br>
1) 'vlogCountAll' - number of vlogs posted in lifetime of channel<br>
2) 'vlogCountBegin' - number of vlogs posted in first year of channel<br>
3) 'vlogCount2015' - number of vlogs posted in 2015<br>
4) 'vlogCount2016' - number of vlogs posted in 2016<br>
5) 'vlogCount2017' - number of vlogs posted in 2017<br>

In [36]:
channels=list(channelData['channelTitle'].unique())

channelData['vlogCountAll']=0
channelData['vlogCountBegin']=0
channelData['vlogCount2015']=0
channelData['vlogCount2016']=0
channelData['vlogCount2017']=0

channelData['vlogCountAll']=channelData['vlogCountAll'].astype(float)
channelData['vlogCountBegin']=channelData['vlogCountBegin'].astype(float)
channelData['vlogCount2015']=channelData['vlogCount2015'].astype(float)
channelData['vlogCount2016']=channelData['vlogCount2016'].astype(float)
channelData['vlogCount2017']=channelData['vlogCount2017'].astype(float)


row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()
    
    channelData['vlogCountAll'][row]=df_all[df_all['vlog']=='yes'].shape[0]
    channelData['vlogCountBegin'][row]=df_begin[df_begin['vlog']=='yes'].shape[0]
    
    if channelData.yearBegan[i]<=2015:
        channelData['vlogCount2015'][row]=df_2015[df_2015['vlog']=='yes'].shape[0]
    
    if channelData.yearBegan[i]<=2016:
        channelData['vlogCount2016'][row]=df_2016[df_2016['vlog']=='yes'].shape[0]
        
    if channelData.yearBegan[i]<=2017:
        channelData['vlogCount2017'][row]=df_2017[df_2017['vlog']=='yes'].shape[0]
    
channelData.head(5)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017,channelSize2,plistCountAll,plistCountBegin,plistCount2015,plistCount2016,plistCount2017,vlogCountAll,vlogCountBegin,vlogCount2015,vlogCount2016,vlogCount2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285,8.380000,8.380000,0.000000,0.000000,8.38000,large,17.0,17.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304,6.308917,5.811321,5.180000,5.750000,6.67000,large,21.0,21.0,21.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0
2,5-Minute Crafts,2016,1,150,50,0,50,100,26082766,26082766,0.0,313697.0,26082766.0,2409232,2409232,0.0,59643.0,3584027.0,3524384,0,0,3524384,growing,same,same,growing,1,2,2,1,14109560,0.907235,0.066111,0.052616,0.807436,0.040443,0.031656,0.000000,0.000000,0.000000,0.807436,0.040443,0.031656,0.957135,0.078944,0.063096,11.566667,0.500000,0.000000,0.500000,17.10000,large,7.0,7.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0
3,Jaclyn Hill,2011,6,301,4,60,48,32,15429406,15429406,6042557.0,6277132.0,9018504.0,1312493,1312493,1532521.0,1624083.0,2155248.0,1803220,523383,91561,531164,growing,growing,growing,growing,1,1,1,1,4919905,3.343569,0.068461,0.412224,1.691926,0.028710,0.150864,3.920757,0.058560,0.530401,4.372235,0.092529,0.606775,4.611552,0.172237,0.566143,14.631229,6.000000,16.016667,17.458333,20.65625,large,14.0,14.0,14.0,14.0,14.0,6.0,0.0,0.0,1.0,0.0
4,Rosanna Pansino,2011,6,523,13,91,96,100,176018933,176018933,51699304.0,20856918.0,13306227.0,4297539,4297539,6056932.0,3253271.0,1720934.0,274088,-1394894,-2803660,-1532337,growing,shrinking,shrinking,shrinking,1,0,0,0,9643380,1.736919,0.031885,0.399245,0.825019,0.034042,0.182375,1.611941,0.029157,0.411249,2.291382,0.029794,0.471068,3.108589,0.040637,0.572232,8.005736,4.923077,8.164835,8.354167,7.77000,large,5.0,5.0,5.0,5.0,5.0,4.0,0.0,0.0,1.0,0.0


## Add in info on time of posting (weekend/weekday)

Assign numerical value for weekday or weekend. This will allow us to construct an average (between 0 and 1) to determine if videos are more frequently posted on weekends or weekdays.

In [37]:
def det_type_of_day2(data):
    if data=='weekday':
        return 0
    if data=='weekend':
        return 1

data['type_of_day2']=data['type_of_day'].apply(func=det_type_of_day2)
data.head(1)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length,series,description_length,posInteract,negInteract,anyInteract,comInteract,type_of_day2
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45,no,199,2.098313,0.093789,2.192102,0.230358,0


Add following variable:<br>
1) 'dayofweekBegin' - average day that videos are posted in lifetime of channel<br>
2) 'dayofweekAll' - average day that videos are posted in first year of channel<br>
3) 'dayofweek2015' - average day that videos are posted in 2015<br>
4) 'dayofweek2016' - average day that videos are posted in 2016<br>
5) 'dayofweek2017' - average day that videos are posted in 2017<br>

average closer to 0, indicates that videos were more frequently posted on a weekday<br>
average closer to 1, indicates that videos were more frequently posted on a weekend

In [38]:
channelData['dayofweekBegin']=0
channelData['dayofweekAll']=0
channelData['dayofweek2015']=0
channelData['dayofweek2016']=0
channelData['dayofweek2017']=0

channelData['dayofweekBegin']=channelData['dayofweekBegin'].astype(float)
channelData['dayofweekAll']=channelData['dayofweekAll'].astype(float)
channelData['dayofweek2015']=channelData['dayofweek2015'].astype(float)
channelData['dayofweek2016']=channelData['dayofweek2016'].astype(float)
channelData['dayofweek2017']=channelData['dayofweek2017'].astype(float)

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()

    channelData['dayofweekBegin'][row]=df_begin['type_of_day2'].mean()
    channelData['dayofweekAll'][row]=df_all['type_of_day2'].mean()
    
    if channelData.yearBegan[i]<=2015:
        channelData['dayofweek2015'][row]=df_2015['type_of_day2'].mean()
        
    if channelData.yearBegan[i]<=2016:  
        channelData['dayofweek2016'][row]=df_2016['type_of_day2'].mean()
        
    if channelData.yearBegan[i]<=2017:  
        channelData['dayofweek2017'][row]=df_2017['type_of_day2'].mean()

channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017,channelSize2,plistCountAll,plistCountBegin,plistCount2015,plistCount2016,plistCount2017,vlogCountAll,vlogCountBegin,vlogCount2015,vlogCount2016,vlogCount2017,dayofweekBegin,dayofweekAll,dayofweek2015,dayofweek2016,dayofweek2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285,8.380000,8.380000,0.00,0.00,8.38,large,17.0,17.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.280000,0.28000,0.00,0.00,0.28
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304,6.308917,5.811321,5.18,5.75,6.67,large,21.0,21.0,21.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0,0.830189,0.52707,0.35,0.33,0.34


## Add in info on time of posting (morning or afternoon)

Assign numerical value for morning or afternoon. This will allow us to construct an average (between 0 and 1) to determine if videos are more frequently posted in the morning or afternoon.

In [39]:
def det_timeofday(data):
    if data=='morning':
        return 0
    if data=='afternoon':
        return 1

data['timeofday2']=data['timeofday'].apply(func=det_timeofday)
data.head(1)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,month_name,dayofweek,dayofweek2,season,type_of_day,hour,timeofday,vlog,vlogmas,title_length,series,description_length,posInteract,negInteract,anyInteract,comInteract,type_of_day2,timeofday2
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,0,False,7860308,894,1523479095,17,1.0,0,5,47,5,347,2017,6,26,29,180,Jun,3,Thurs,summer,weekday,17,afternoon,no,no,45,no,199,2.098313,0.093789,2.192102,0.230358,0,1


Add following variable:<br>
1) 'timeofdayBegin' - average time that videos are posted in lifetime of channel<br>
2) 'timeofdayAll' - average time that videos are posted in first year of channel<br>
3) 'timeofday2015' - average time that videos are posted in 2015<br>
4) 'timeofday2016' - average time that videos are posted in 2016<br>
5) 'timeofday2017' - average time that videos are posted in 2017<br>

average closer to 0, indicates that videos were more frequently posted in the morning<br>
average closer to 1, indicates that videos were more frequently posted in the afternoon

In [40]:
channelData['timeofdayBegin']=0
channelData['timeofdayAll']=0
channelData['timeofday2015']=0
channelData['timeofday2016']=0
channelData['timeofday2017']=0

channelData['timeofdayBegin']=channelData['timeofdayBegin'].astype(float)
channelData['timeofdayAll']=channelData['timeofdayAll'].astype(float)
channelData['timeofday2015']=channelData['timeofday2015'].astype(float)
channelData['timeofday2016']=channelData['timeofday2016'].astype(float)
channelData['timeofday2017']=channelData['timeofday2017'].astype(float)

row=-1
for i in range(channelData.channelTitle.shape[0]):
    row=row+1
    channel=channelData.channelTitle[i]
    df_all   =data[data['channelTitle']==channel].reset_index()
    df_begin =df_all[df_all['year']==channelData.yearBegan[i]].reset_index()
    df_2015  =df_all[df_all['year']==2015].reset_index()
    df_2016  =df_all[df_all['year']==2016].reset_index()
    df_2017  =df_all[df_all['year']==2017].reset_index()

    channelData['timeofdayBegin'][row]=df_begin['timeofday2'].mean()
    channelData['timeofdayAll'][row]=df_all['timeofday2'].mean()
    
    if channelData.yearBegan[i]<=2015:
        channelData['timeofday2015'][row]=df_2015['timeofday2'].mean()
        
    if channelData.yearBegan[i]<=2016:  
        channelData['timeofday2016'][row]=df_2016['timeofday2'].mean()

    if channelData.yearBegan[i]<=2017:  
        channelData['timeofday2017'][row]=df_2017['timeofday2'].mean()

channelData.head(2)

,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,posInteractBegin,negInteractBegin,comInteractBegin,posInteract2015,negInteract2015,comInteract2015,posInteract2016,negInteract2016,comInteract2016,posInteract2017,negInteract2017,comInteract2017,minsTotalAll,minsTotalBegin,minsTotal2015,minsTotal2016,minsTotal2017,channelSize2,plistCountAll,plistCountBegin,plistCount2015,plistCount2016,plistCount2017,vlogCountAll,vlogCountBegin,vlogCount2015,vlogCount2016,vlogCount2017,dayofweekBegin,dayofweekAll,dayofweek2015,dayofweek2016,dayofweek2017,timeofdayBegin,timeofdayAll,timeofday2015,timeofday2016,timeofday2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7860308,1.881192,0.138871,0.227285,1.881192,0.138871,0.227285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.881192,0.138871,0.227285,8.380000,8.380000,0.00,0.00,8.38,large,17.0,17.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.280000,0.28000,0.00,0.00,0.28,0.380000,0.380000,0.00,0.00,0.38
1,ATHLEAN-X™,2010,7,628,53,100,100,100,13523207,13523207,7384932.0,9012192.0,6088234.0,751737,751737,836367.0,849244.0,842108.0,478717,-118244,12877,-7136,growing,shrinking,growing,shrinking,1,0,1,0,4612450,2.395049,0.037721,0.165887,0.723130,0.020664,0.086658,2.576613,0.040476,0.143578,3.353887,0.039008,0.213624,3.951970,0.051145,0.263304,6.308917,5.811321,5.18,5.75,6.67,large,21.0,21.0,21.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0,0.830189,0.52707,0.35,0.33,0.34,0.339623,0.347134,0.13,0.11,0.31


In [5]:
#Save data
#data.to_csv('Youtube_Part2_cat26_byview_2017_AllData_Final.csv')
data=pd.read_csv('Youtube_Part2_cat26_byview_2017_AllData_Final.csv')
del data['Unnamed: 0']
data.head(1)

#channelData.to_csv('Youtube_Part2_cat26_byview_2017_ChannelData_Final.csv')
channelData=pd.read_csv('Youtube_Part2_cat26_byview_2017_ChannelData_Final.csv')
del channelData['Unnamed: 0']
channelData.head(1)

,dateDownloaded,channelId,channelTitle,kind,videoId,title,description,publishedAt,vidfrequency,vidtype,categoryId,videoDuration,commentCount,dislikeCount,likeCount,viewCount,posInteract,negInteract,anyInteract,comInteract,hours,minutes,seconds,total_mins,total_secs,year,month,week,day,dayofyear,vlog,vlogmas,title_length,description_length,title_channel,month_name,season,dayofweek,dayofweek2,type_of_day,hour,timeofday,series,stats_commentCount,stats_hiddenSubscriberCount,stats_subscriberCount,stats_videoCount,stats_viewCount,num_of_playlists,num_of_playlists_forvid
0,2018-02-17T15:32:34,UC4rlAVgAK0SGk-yTfe48Qpw,BRIGHT SIDE,youtube#video,Pb8RAH4ywNY,10 Signs You're STUCK in a Toxic Relationship,Subscribe to Bright Side : https://goo.gl/rQTJ...,2017-06-29T17:00:02.000Z,110,26,26,PT5M47S,560.0,228.0,5101.0,243100.0,2.098313,0.093789,2.192102,0.230358,0,5,47,5,347,2017,6,26,29,180,no,no,45,199,excludes_CT,Jun,summer,3,Thurs,weekday,17,afternoon,no,0,False,7648588,873,1480889507,17,1.0


,channelTitle,yearBegan,yearCount,vidCountAll,vidCountBegin,vidCount2015,vidCount2016,vidCount2017,viewMaxAll,viewMaxBegin,viewMax2015,viewMax2016,viewMax2017,viewAvgAll,viewAvgBegin,viewAvg2015,viewAvg2016,viewAvg2017,viewChangeAll,viewChange2015,viewChange2016,viewChange2017,viewLabelAll,viewLabel2015,viewLabel2016,viewLabel2017,viewLabelAll_2,viewLabel2015_2,viewLabel2016_2,viewLabel2017_2,stats_subscriberCount,posInteractAll,negInteractAll,comInteractAll,minsTotalAll,posInteractBegin,negInteractBegin,comInteractBegin,minsTotalBegin,posInteract2015,negInteract2015,comInteract2015,minsTotal2015,posInteract2016,negInteract2016,comInteract2016,minsTotal2016,posInteract2017,negInteract2017,comInteract2017,minsTotal2017,channelSize2,vlogCountAll,vlogCountBegin,vlogCount2015,vlogCount2016,vlogCount2017,dayofweekBegin,dayofweekAll,dayofweek2015,dayofweek2016,dayofweek2017,timeofdayBegin,timeofdayAll,timeofday2015,timeofday2016,timeofday2017
0,BRIGHT SIDE,2017,0,100,100,0,0,100,17909419,17909419,0.0,0.0,17909419.0,1611841,1611841,0.0,0.0,1611841.0,0,0,0,0,same,same,same,same,2,2,2,2,7648588,1.881192,0.138871,0.227285,8.38,1.881192,0.138871,0.227285,8.38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.881192,0.138871,0.227285,8.38,large,0.0,0.0,0.0,0.0,0.0,0.28,0.28,0.0,0.0,0.28,0.38,0.38,0.0,0.0,0.38
